# Bilby 4 param BBH tutorial

Tutorial to demonstrate running parameter estimation on a reduced parameter
space for an injected binary black hole signal.





## Fixed parameters

1. Extrinsic
    1. Polarization angle `psi`, 
    1. Sky location `ra`, `dec`,
    1. Signal's arrival time at geocenter `geocent_time`,
    1. Orbital / coalescence `phase`,
1. Intrinsic
    1. Spin components `a_1`, `a_2`, `tilt_1`, `tilt_2`, `phi_12`, `phi_jl`

##  To sample over
1. Intrinsic:
    1. Component masses `m_1`, `m_2` in terms of `mass_ratio`, `chirp_mass`
    
    \begin{equation}
     q = \dfrac{m_1}{m_2} \qquad,\,\, \mathcal{M} = \dfrac{(m_1 * m_2)^{3/5}}{(m_1 + m_2)^{1/5}}
    \end{equation}

1. Extrinsic:
    1. Luminosity distance `luminosity_distance`
    1. Inclination angle `theta_jn`


## Priors
1. Uniform in (component masses)
1. Uniform in comoving volume prior on luminosity distance (100Mpc, 5Gpc)
1. 


## Available samplers

https://bilby-dev.github.io/bilby/samplers.html

## Collab setup

In [ ]:
!pip install pycbc lalsuite bilby nestle matplotlib

## PC setup

### Using anaconda/miniconda
`conda env create -f environment.yaml`

`conda activate np3m`

OR

### Using pip
Create a python virtual environment and 
`pip install -r requirements.txt`

In [ ]:
import bilby

# Set the duration and sampling frequency of the data segment that we're
# going to inject the signal into
duration = 4.0
sampling_frequency = 2048.0
minimum_frequency = 20

# Specify the output directory and the name of the simulation.
outdir = "bbh_outdir"
label = "4p_tutorial"

## Injection parameters

In [ ]:

bilby.core.utils.setup_logger(outdir=outdir, 
                              label=label, 
                              log_level="DEBUG")

# Set up a random seed for result reproducibility.  This is optional!
bilby.core.utils.random.seed(88170235)

# We are going to inject a binary black hole waveform.  We first establish a
# dictionary of parameters that includes all of the different waveform
# parameters, including masses of the two black holes (mass_1, mass_2),
# spins of both black holes (a, tilt, phi), etc.
injection_parameters = dict(
    mass_1=36.0,
    mass_2=29.0,
    a_1=0.4,
    a_2=0.3,
    tilt_1=0.5,
    tilt_2=1.0,
    phi_12=1.7,
    phi_jl=0.3,
    luminosity_distance=2000.0,
    theta_jn=0.4,
    psi=2.659,
    phase=1.3,
    geocent_time=1126259642.413,
    ra=1.375,
    dec=-1.2108,
)

m1 = injection_parameters["mass_1"]
m2 = injection_parameters["mass_2"]
inj_mass_ratio = injection_parameters["mass_2"]/injection_parameters["mass_1"]

inj_chirp_mass = ((m1*m2)**(3/5)) /(m1+m2)**(1/5)

In [ ]:
truths = {"mass_ratio" : inj_mass_ratio,
          "chirp_mass" : inj_chirp_mass,
          "luminosity_distance": injection_parameters["luminosity_distance"],
          "theta_jn" : injection_parameters["theta_jn"]}

# Waveform generator

In [ ]:

# Fixed arguments passed into the source model
waveform_arguments = dict(
    waveform_approximant="IMRPhenomXPHM",
    reference_frequency=50.0,
    minimum_frequency=minimum_frequency,
)

# Create the waveform_generator using a LAL BinaryBlackHole source function
waveform_generator = bilby.gw.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments=waveform_arguments,
)

## Inject the signal into the detector

In [ ]:
# Set up interferometers.  In this case we'll use two interferometers
# (LIGO-Hanford (H1), LIGO-Livingston (L1). These default to their design
# sensitivity
ifos = bilby.gw.detector.InterferometerList(["H1", "L1"])
ifos.set_strain_data_from_power_spectral_densities(
    sampling_frequency=sampling_frequency,
    duration=duration,
    start_time=injection_parameters["geocent_time"] - 2,
)



ifos.inject_signal(
    waveform_generator=waveform_generator, parameters=injection_parameters
)

# Set up a PriorDict, which inherits from dict.
# By default we will sample all terms in the signal models.  However, this will
# take a long time for the calculation, so for this example we will set almost
# all of the priors to be equall to their injected values.  This implies the
# prior is a delta function at the true, injected value.  In reality, the
# sampler implementation is smart enough to not sample any parameter that has
# a delta-function prior.
# The above list does *not* include mass_1, mass_2, theta_jn and luminosity
# distance, which means those are the parameters that will be included in the
# sampler.  If we do nothing, then the default priors get used.


## Define the priors

In [ ]:
priors = bilby.gw.prior.BBHPriorDict()
for key in [
    "a_1",
    "a_2",
    "tilt_1",
    "tilt_2",
    "phi_12",
    "phi_jl",
    "psi",
    "ra",
    "dec",
    "geocent_time",
    "phase",
#    "theta_jn"
]:
    priors[key] = injection_parameters[key]

# Perform a check that the prior does not extend to a parameter space longer than the data
priors.validate_prior(duration, minimum_frequency)

## Initialize the likelihood

In [ ]:
# Initialise the likelihood by passing in the interferometer data (ifos) and
# the waveform generator
likelihood = bilby.gw.GravitationalWaveTransient(
    interferometers=ifos, waveform_generator=waveform_generator
)

## Run the sampler

In [ ]:

# Run sampler.  In this case we're going to use the `dynesty` sampler
result = bilby.run_sampler(
    likelihood=likelihood,
    priors=priors,
    sampler="dynesty",
    npoints=100,
    injection_parameters=injection_parameters,
    outdir=outdir,
    label=label,
    result_class=bilby.gw.result.CBCResult,
)

## Plot the results

In [ ]:
# Make a corner plot.
result.plot_corner(truths=truths)